**Model Training**
   - **Multinomial Logistic Regression Forecasting Model**:
     - Split the data into training and testing sets.
     - Build and compile a neural network model for multinomial logistic regression using TensorFlow.
     - Train the model on the training set.
 **Evaluation and Scoring**
   - Evaluate the model on the test set and visualize the results.
       
   
### Goal
The objective of this project is to develop a Multinomial Logistic Regression model that predicts a person's vision status (Normal Vision, Visual Impairment, Blindness) based on demographic and health-related factors

**Example**:
#### **Input:**
| Age  | Gender | RiskFactor (Diabetes) | RiskFactor (Smoking) | RiskFactorResponse (Hypertension) |
|------|--------|-----------------------|----------------------|-----------------------------------|
| 50   | Male   | Yes                   | No                   | Yes                               |

#### **Output (Vision Status Prediction)**:
| Vision Status | Probability   |
|---------------|---------------|
| Normal vision | 0.60          |
| Visual impairment | 0.25      |
| Blindness     | 0.15          |

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from IPython.display import display

### Load the Data

In [2]:
df = pd.read_csv('data/Vision_Survey_Cleaned.csv')
"""
    Filter the dataset to include only the relevant columns for the model.
"""
vision_cat = ['Best-corrected visual acuity']
df.drop(df[~df['Question'].isin(vision_cat)].index, inplace=True)

df.drop(df[df["RiskFactor"] == "All participants"].index, inplace=True)
df.drop(df[df["RiskFactorResponse"] == "Total"].index, inplace=True)
print("First few rows of the dataset:")
display(df.head())

First few rows of the dataset:


,Category,Question,Response,Age,Gender,RaceEthnicity,RiskFactor,RiskFactorResponse,Sample_Size
0,Measured Visual Acuity,Best-corrected visual acuity,Visual impairment,40-64 years,All genders,Other,Smoking,Yes,155
1,Measured Visual Acuity,Best-corrected visual acuity,US-defined blindness,12-17 years,Male,Other,Diabetes,No,54
3,Measured Visual Acuity,Best-corrected visual acuity,Any vision loss,18-39 years,Female,All races,Smoking,Yes,1511
4,Measured Visual Acuity,Best-corrected visual acuity,Any vision loss,40-64 years,All genders,Other,Diabetes,No,130
5,Measured Visual Acuity,Best-corrected visual acuity,Monocular vision loss,65-79 years,All genders,Other,Diabetes,Yes,37


### Generate Test and Train Datasets


In [3]:
"""
Define the features set X by selecting all the columns but "Vision Status" from the DataFrame
"""

X = df[["Age", "RiskFactor", "RiskFactorResponse", "Gender", "RaceEthnicity"]]
Y = df["Response"]

total_samples = df['Sample_Size'].sum()
sample_weights = (df['Sample_Size'] / total_samples) * 100

print(sample_weights)

categorical_mappings = {
    col: list(X[col].unique()) for col in X.columns if X[col].dtype == 'object'
}

0       0.011309
1       0.003940
3       0.110241
4       0.009485
5       0.002699
          ...   
4998    0.138987
5000    0.350277
5007    0.076315
5014    0.016489
5016    0.035385
Name: Sample_Size, Length: 1729, dtype: float64


### Convert Categorical Data to Numeric

In [4]:
"""
    Convert categorical data to numeric using one-hot encoding.
"""

X = pd.get_dummies(X)
print("Features after encoding:")
display(X.head())

Features after encoding:


,Age_12-17 years,Age_18-39 years,Age_40-64 years,Age_65-79 years,Age_80 years and older,RiskFactor_Diabetes,RiskFactor_Hypertension,RiskFactor_Smoking,RiskFactorResponse_No,RiskFactorResponse_Yes,Gender_All genders,Gender_Female,Gender_Male,RaceEthnicity_All races,"RaceEthnicity_Black, non-Hispanic","RaceEthnicity_Hispanic, any race",RaceEthnicity_Other,"RaceEthnicity_White, non-Hispanic"
0,False,False,True,False,False,False,False,True,False,True,True,False,False,False,False,False,True,False
1,True,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,True,False
3,False,True,False,False,False,False,False,True,False,True,False,True,False,True,False,False,False,False
4,False,False,True,False,False,True,False,False,True,False,True,False,False,False,False,False,True,False
5,False,False,False,True,False,True,False,False,False,True,True,False,False,False,False,False,True,False


### Split the Data into Training and Testing Sets

In [5]:
"""
    Split the data into training and testing sets.
"""

X_train, X_test, Y_train, Y_test, weights_train, weights_test = train_test_split(
    X, Y, sample_weights, random_state=1, test_size=0.2
)

print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (1383, 18)
Testing set shape: (346, 18)


### Evaluate the Model

In [6]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Define parameter grids for each model
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_grid_svc = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

param_grid_lr = {
    'C': [0.1, 1, 10],
    'penalty': ['l2', 'elasticnet', 'l1'],
    'solver': ['saga'],
    'max_iter': [100, 200],
    'l1_ratio': [0.1, 0.5, 0.9]
}

# Initialize models
rf = RandomForestClassifier()
svc = SVC()
lr = LogisticRegression(max_iter=200)

# Perform Grid Search for RandomForestClassifier
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, n_jobs=-1, verbose=2)
grid_search_rf.fit(X_train, Y_train, sample_weight=weights_train)
best_rf = grid_search_rf.best_estimator_

# Perform Grid Search for Support Vector Classifier
grid_search_svc = GridSearchCV(estimator=svc, param_grid=param_grid_svc, cv=5, n_jobs=-1, verbose=2)
grid_search_svc.fit(X_train, Y_train, sample_weight=weights_train)
best_svc = grid_search_svc.best_estimator_

grid_search_lr = GridSearchCV(estimator=lr, param_grid=param_grid_lr, cv=5, n_jobs=-1, verbose=2)
grid_search_lr.fit(X_train, Y_train, sample_weight=weights_train)
best_lr = grid_search_lr.best_estimator_

# Create a list of models to evaluate
models = [
    ("Random Forest", best_rf),
    ("SVM", best_svc),
    ("Logistic Regression", best_lr)
]

# Evaluate each model
for name, model in models:
    try:
        y_pred = model.predict(X_test)
        print(f"Model: {name}")
        print(f"Accuracy: {accuracy_score(Y_test, y_pred):.2f}")
        print(f"F1 Score: {f1_score(Y_test, y_pred, average='weighted'):.2f}\n")
    except Exception as e:
        print(f"Error occurred while evaluating {name}: {e}")

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Fitting 5 folds for each of 54 candidates, totalling 270 fits
Model: Random Forest
Accuracy: 0.01
F1 Score: 0.01

Model: SVM
Accuracy: 0.23
F1 Score: 0.08

Model: Logistic Regression
Accuracy: 0.23
F1 Score: 0.08



C:\Users\Anwender\PycharmProjects\DataScienceDataCleaning\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(


## Make custom input 

In [7]:
import joblib

def save_model(model, path='models/vision_status_model.pkl'):
    """
    :param model: 
    :param path: 
    :return: 
    
    Save the model to disk.
    set configuration for the model and save it to disk.
    """
    model_data = {
        'model': model,
        'feature_columns': X.columns.tolist(),
    }
    
    # Save the model to disk
    joblib.dump(model_data, path)
    
save_model(best_lr)


## Write method to predict vision status

In [8]:
def predict_vision_status(input_data, model_path='models/vision_status_model.pkl'):
    """
    Make predictions using the saved model.

    Args:
        input_data (dict): Dictionary containing input features.
        model_path (str): Path to the saved model.

    Returns:
        dict: Predicted probabilities for each vision status.
    """
    try:
        # Load the model directly
        model = joblib.load(model_path)
         
        # Specify feature columns manually (temporary workaround)
        feature_columns = model['feature_columns']
        
        # Convert input data to DataFrame
        input_df = pd.DataFrame([input_data])

        # Encode categorical variables
        input_encoded = pd.get_dummies(input_df)

        # Ensure all expected feature columns are present
        for col in feature_columns:
            if col not in input_encoded.columns:
                input_encoded[col] = 0

        # Reorder columns to match the model's training format
        input_encoded = input_encoded[feature_columns]

        # Predict probabilities
        probabilities = model['model'].predict_proba(input_encoded)[0]

        # Create results dictionary with readable format
        results = {
            'predictions': [
                {'status': status, 'probability': float(prob)}
                for status, prob in zip(model['model'].classes_, probabilities)
            ]
        }
        
        return results

    except Exception as e:
        raise ValueError(f"Error making prediction: {str(e)}")


### Make a Test Method


In [9]:
def test_prediction():
    # First save the model with the correct structure
    sample_input = {
        "Age": "18-39 years",
        "Gender": "Male",
        "RiskFactor_Diabetes": "Yes",
        "RiskFactor_Smoking": "Yes",
        "RiskFactor_Hypertension": "Yes",
    }
    
    # Make prediction
    try:
        predictions = predict_vision_status(sample_input)
        display(predictions)
        print("\nVision Status Predictions:")
        for pred in predictions['predictions']:
            print(f"{pred['status']}: {pred['probability']:.2%}")
        return True
    except Exception as e:
        print(f"{str(e)}")
        return False

## Evaluate custom input


In [10]:
test_prediction()

{'predictions': [{'status': 'Any vision loss',
   'probability': 0.19556518149660812},
  {'status': 'Monocular vision loss', 'probability': 0.20862938571141865},
  {'status': 'Normal vision', 'probability': 0.1845282213010377},
  {'status': 'US-defined blindness', 'probability': 0.20763345802786176},
  {'status': 'Visual impairment', 'probability': 0.20364375346307384}]}


Vision Status Predictions:
Any vision loss: 19.56%
Monocular vision loss: 20.86%
Normal vision: 18.45%
US-defined blindness: 20.76%
Visual impairment: 20.36%


True

### 
